# Homo NN MNIST图像分类: 介绍数据集与模型自定义

在该版本中 整个nn的架构有很大的调整，nn模块开发了dataset与model_zoo模块，旨在提供数据集和模型的自定义功能，在这个教程中， 我们将会介绍nn的Dataset机制，以及介绍如何进行模型自定义

## Dataset

在 FATE-1.10中，FATE基于pytorch的Dataset类开发一个新的Dataset基类，用户可以基于这个类，实现\_\_getitem\_\_, \_\_len\_\_, load接口，
load接口接收一个地址，可以从本地读取数据。当提交任务时，可以通过reader组件输入数据地址，HomoNN会使用用户指定的Dataset，使用load接口读取数据，使用数据集进行训练

在FATE-1.10中，提供了table, nlp_tokenizer, image三种数据集，以满足基本的使用需要，但是用户可以随意自定义自己的dataset，dataset的自定义将会在下一教程介绍

本次教程，我们介绍image数据集，它基于pytorch的ImageFolder类开发

### ImageDataset
此处 我们用ImageDataset来加载MNIST数据

In [1]:
from federatedml.nn.dataset.image import ImageDataset

In [2]:
# 查看mnist数据集文件夹结构，一共10个文件夹，每个文件夹下放有各个类图像，使用方式与ImageFolder一致
! ls ../examples/data/mnist_folder/

0  1  2  3  4  5  6  7	8  9


In [3]:
dataset = ImageDataset()
dataset.load('../examples/data/mnist/') # 读取mnist数据

In [4]:
len(dataset)

1309

In [5]:
dataset[400] # 查看图像 与ImageFolder使用方式一致

(tensor([[[0.0000, 0.0275, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0118, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
 
         [[0.0000, 0.0275, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0118, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
 
         [[0.0000, 0.0275, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0118, 0.0000, 0.0000,  ...,

## 自定义模型

在 FATE-1.10中，FATE可以通过pipeline提交pytorch Sequential模型.然而，Sequential结合pytorch自带的layer, 还是无法表示更为复杂的模型

因此，FATE-1.10中加入了model_zoo模块，位于federatedml.nn.model_zoo下。现在，你可以定制自己的pytorch模型，**前提是这个模型必须是基于t.nn.Module开发，并实现了forward接口**. 将你实现好的模型文件放入federatedml/nn/model_zoo下, 在提交任务时通过接口指定该模块，homo-nn会自动搜寻并导入你实现的模型

这里，为了完成MNIST分类任务，我们先本地编写一个带有卷积网络的NN模块:

In [6]:
from torch import nn
import torch as t
from torch.nn import functional as F

class ImgNet(nn.Module):
    def __init__(self, class_num=10):
        super(ImgNet, self).__init__()
        self.seq = t.nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=12, kernel_size=5),
            nn.MaxPool2d(kernel_size=3),
            nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3),
            nn.AvgPool2d(kernel_size=3)
        )
        
        self.fc = t.nn.Sequential(
            nn.Linear(48, 32),
            nn.ReLU(),
            nn.Linear(32, class_num)
        )
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.seq(x)
        x = x.flatten(start_dim=1)
        x = self.fc(x)
        if self.training:
            return x
        else:
            return self.softmax(x)

In [7]:
img_model = ImgNet(10)
img_model

ImgNet(
  (seq): Sequential(
    (0): Conv2d(3, 12, kernel_size=(5, 5), stride=(1, 1))
    (1): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1))
    (3): AvgPool2d(kernel_size=3, stride=3, padding=0)
  )
  (fc): Sequential(
    (0): Linear(in_features=48, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=10, bias=True)
  )
  (softmax): Softmax(dim=1)
)

将模型代码命名为image_net.py，可以直接把它放在federatedml/nn/model_zoo下

或者使用jupyter notebook的快捷接口 直接将其保存到federatedml/nn/model_zoo

In [8]:
from pipeline.component.homo_nn import save_to_fate

## jupyter保存接口

In [9]:
%%save_to_fate model image_net.py
from torch import nn
import torch as t
from torch.nn import functional as F

class ImgNet(nn.Module):
    def __init__(self, class_num=10):
        super(ImgNet, self).__init__()
        self.seq = t.nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=12, kernel_size=5),
            nn.MaxPool2d(kernel_size=3),
            nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3),
            nn.AvgPool2d(kernel_size=3)
        )
        
        self.fc = t.nn.Sequential(
            nn.Linear(48, 32),
            nn.ReLU(),
            nn.Linear(32, class_num)
        )
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.seq(x)
        x = x.flatten(start_dim=1)
        x = self.fc(x)
        if self.training:
            return x
        else:
            return self.softmax(x)

In [10]:
! cat ../fate/python/federatedml/nn/model_zoo/image_net.py

from torch import nn
import torch as t
from torch.nn import functional as F

class ImgNet(nn.Module):
    def __init__(self, class_num=10):
        super(ImgNet, self).__init__()
        self.seq = t.nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=12, kernel_size=5),
            nn.MaxPool2d(kernel_size=3),
            nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3),
            nn.AvgPool2d(kernel_size=3)
        )
        
        self.fc = t.nn.Sequential(
            nn.Linear(48, 32),
            nn.ReLU(),
            nn.Linear(32, class_num)
        )
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.seq(x)
        x = x.flatten(start_dim=1)
        x = self.fc(x)
        if self.training:
            return x
        else:
            return self.softmax(x)


## 进行本地测试

我们可以先不急着提交任务，可以使用Trainer进行本地测试。上个教程提到，fate自带一个FedAVGTrainer

我们可以使用我们的数据集，自定义的模型，和Trainer进行本地调试，测试是否能够跑通程序
**本地测试的情况下，会跳过所有联邦流程，模型不会进行fed averaging**

In [11]:
from federatedml.nn.homo.trainer.fedavg_trainer import FedAVGTrainer
trainer = FedAVGTrainer(epochs=3, batch_size=256, shuffle=True, data_loader_worker=8) # 参数
trainer.set_model(img_model) # 设置自定义模型

In [12]:
trainer.local_mode() # !! 请务必启用local_mode以跳过联邦流程 !!

In [13]:
optimizer = t.optim.Adam(img_model.parameters(), lr=0.01)
loss = t.nn.CrossEntropyLoss()
trainer.train(train_set=dataset,optimizer=optimizer, loss=loss)

epoch is 0
epoch loss is 2.280080816780364
epoch is 1
epoch loss is 2.0551508919167283
epoch is 2
epoch loss is 1.5070739016449115


可以跑通！那我们可以提交联邦任务了

## 提交使用自定义模型的Homo-NN任务

In [14]:
import torch as t
from torch import nn
from pipeline import fate_torch_hook
from pipeline.component import HomoNN
from pipeline.backend.pipeline import PipeLine
from pipeline.component import Reader, Evaluation, DataTransform
from pipeline.interface import Data, Model

fate_torch_hook(t)


<module 'torch' from '/home/cwj/standalone_fate_install_1.9.0_release/env/python/venv/lib/python3.8/site-packages/torch/__init__.py'>

In [15]:
import os
# 绑定地址到fate name&namespace
fate_project_path = os.path.abspath('../')
host_0 = 10000
host_1 = 9999
pipeline = PipeLine().set_initiator(role='host', party_id=host_0).set_roles(host=[host_0, host_1],
                                                                            arbiter=[host_0])

data_0 = {"name": "mnist_host_0", "namespace": "experiment"}
data_1 = {"name": "mnist_host_1", "namespace": "experiment"}

# 为方便，本示例中两个client使用同一份数据集
data_path_0 = fate_project_path + '/examples/data/mnist'
data_path_1 = fate_project_path + '/examples/data/mnist'
pipeline.bind_table(name=data_0['name'], namespace=data_0['namespace'], path=data_path_0)
pipeline.bind_table(name=data_1['name'], namespace=data_1['namespace'], path=data_path_1)

{'namespace': 'experiment', 'table_name': 'mnist_host_1'}

In [16]:
# 定义reader
reader_0 = Reader(name="reader_0")
reader_0.get_party_instance(role='host', party_id=host_0).component_param(table=data_0)
reader_0.get_party_instance(role='host', party_id=host_1).component_param(table=data_1)

In [17]:
from pipeline.component.homo_nn import DatasetParam, TrainerParam  # 数据集的接口
from pipeline.component.nn.backend.torch.cust_model import CustModel  # 自定义模型的接口

model = t.nn.Sequential(
    t.nn.CustModel(name='image_net', class_num=10) # name指定自定义模型的模块名，name之外的参数将会被传递到模型的__init__上，比如
                                                    # 此处的class_num
)

nn_component = HomoNN(name='nn_0',
                      model=model, # 模型
                      loss=t.nn.CrossEntropyLoss(),
                      optimizer=t.optim.Adam(model.parameters(), lr=0.01),
                      dataset=DatasetParam(dataset_name='image'),  # 使用image dataset
                      trainer=TrainerParam(trainer_name='fedavg_trainer', epochs=3, batch_size=256, validation_freqs=1),
                      torch_seed=100 # 全局随机种子
                      )

In [18]:
# 添加组件到pipeline，定义数据IO关系，提交即可
pipeline.add_component(reader_0)
pipeline.add_component(nn_component, data=Data(train_data=reader_0.output.data))
pipeline.add_component(Evaluation(name='eval_0', eval_type='multi'), data=Data(data=nn_component.output.data))

In [19]:
pipeline.compile()
pipeline.fit()

2022-11-03 15:46:13.604 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202211031546128971780

2022-11-03 15:46:13.623 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00
m2022-11-03 15:46:14.802 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2022-11-03 15:46:14.803 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:01
2022-11-03 15:46:15.830 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:02
2022-11-03 15:46:16.912 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:03
2022-11-03 15:46:17.954 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:04
2022-11-03 15:46:19.060 | INFO     

2022-11-03 15:46:55.787 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:00:42
2022-11-03 15:46:57.325 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:00:43
2022-11-03 15:46:58.503 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:00:44
2022-11-03 15:46:59.649 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:00:46
2022-11-03 15:47:00.797 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:00:47
2022-11-03 15:47:01.861 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:00:48
2022-11-03 15:47:03.085 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 

成功了！并且 我们可以用同样的方式，为任务加一个验证集，看看效果

## 提交使用自定义模型的Homo-NN任务 + 验证集

In [ ]:
import torch as t
from torch import nn
from pipeline import fate_torch_hook
from pipeline.component import HomoNN
from pipeline.backend.pipeline import PipeLine
from pipeline.component import Reader, Evaluation, DataTransform
from pipeline.interface import Data, Model

fate_torch_hook(t)

import os
# 绑定地址到fate name&namespace
fate_project_path = os.path.abspath('../')
host_0 = 10000
host_1 = 9999
pipeline = PipeLine().set_initiator(role='host', party_id=host_0).set_roles(host=[host_0, host_1],
                                                                            arbiter=[host_0])

validate_data = {"name": "mnist_validate", "namespace": "experiment"}

# 为方便，本示例中两个client使用同一份数据集
data_path_val = fate_project_path + '/examples/data/mnist_val'
pipeline.bind_table(name=validate_data['name'], namespace=validate_data['namespace'], path=data_path_val)

# 定义reader
reader_0 = Reader(name="reader_0")
reader_0.get_party_instance(role='host', party_id=host_0).component_param(table=data_0)
reader_0.get_party_instance(role='host', party_id=host_1).component_param(table=data_1)
# validate data
reader_1 = Reader(name="reader_1")
reader_1.get_party_instance(role='host', party_id=host_0).component_param(table=validate_data)
reader_1.get_party_instance(role='host', party_id=host_1).component_param(table=validate_data)

from pipeline.component.homo_nn import DatasetParam, TrainerParam  # 数据集的接口
from pipeline.component.nn.backend.torch.cust_model import CustModel  # 自定义模型的接口

model = t.nn.Sequential(
    t.nn.CustModel(name='image_net', class_num=10) # name指定自定义模型的模块名，name之外的参数将会被传递到模型的__init__上，比如
                                                    # 此处的class_num
)

nn_component = HomoNN(name='nn_0',
                      model=model, # 模型
                      loss=t.nn.CrossEntropyLoss(),
                      optimizer=t.optim.Adam(model.parameters(), lr=0.01),
                      dataset=DatasetParam(dataset_name='image'),  # 使用image dataset
                      trainer=TrainerParam(trainer_name='fedavg_trainer', epochs=3, batch_size=256, validation_freqs=1),
                      torch_seed=100 # 全局随机种子
                      )

# 添加组件到pipeline，定义数据IO关系，提交即可
pipeline.add_component(reader_0)
pipeline.add_component(reader_1)
pipeline.add_component(nn_component, data=Data(train_data=reader_0.output.data, validate_data=reader_1.output.data))
pipeline.add_component(Evaluation(name='eval_0', eval_type='multi'), data=Data(data=nn_component.output.data))
pipeline.compile()
pipeline.fit()

2022-11-03 15:47:57.092 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202211031547562373210

2022-11-03 15:47:57.137 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00
2022-11-03 15:47:58.154 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:01
m2022-11-03 15:48:00.290 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2022-11-03 15:48:00.293 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:03
2022-11-03 15:48:01.362 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:04
2022-11-03 15:48:02.412 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:05
2022-11-03 15:48:03.745 | INFO     | pipel